In [1]:
import pandas as pd
import os
import re

In [2]:
data_path = '../data/raw/maildir/'
employee_directories = next(os.walk(data_path))[1]

def process_emails(path_to_data):
    emails = []
    for employee in employee_directories:
        employee_path = path_to_data+employee+'/'
        folders = next(os.walk(employee_path))[1]
        if 'inbox' in folders:
            inbox_path = employee_path+'inbox/'
            inbox_emails = next(os.walk(inbox_path))[2]
            inbox_email_data = []
            for file in inbox_emails:
                data = {'Employee': employee, 'Type':'received'}
                try:
                    with open(f'{inbox_path}{file}', 'r') as file:
                        raw_email = file.read()
                        sections = raw_email.split('\n')
                        email_body = []
                        for section in sections:
                            if ': ' in section:
                                field_name = section.split(': ')[0]
                                field_value = section.split(': ')[1]
                                if field_name == 'Date':
                                    data[field_name] = field_value
                                elif field_name == 'X-From':
                                    if '<' in field_value:
                                        sender_name_backwards = field_value.split('<')[0]
                                        if len(sender_name_backwards) == 0:
                                            sender_name = field_value.split('<')[1].split('>')[0]
                                        elif ', ' in sender_name_backwards:
                                            sender_name_list = sender_name_backwards.split(', ')
                                            sender_name = sender_name_list[1] + sender_name_list[0]
                                        elif ',' in sender_name_backwards:
                                            sender_name_list = sender_name_backwards.split(',')
                                            sender_name = sender_name_list[1] + sender_name_list[0]
                                        else:
                                            sender_name = sender_name_backwards
                                        data['From'] = sender_name.replace('"','')
                                    else:
                                        data['From'] = field_value
                                elif field_name == 'From':
                                    if 'From' in data.keys():
                                        pass
                                    else:
                                        sender = field_value.split('@')[0]
                                        if len(sender.split('.')) > 1:
                                            sender_name = sender.split('.')[0].capitalize() + sender.split('.')[1].capitalize()
                                        else:
                                            first_initial = sender[0].upper()
                                            sender_name = first_initial + sender[1:].capitalize()
                                        data['From'] = sender_name
                                elif field_name == 'Subject':
                                    data['Subject'] = field_value
                                else:
                                    continue
                            else:
                                email_body.append(section)
                        email_body = (' ').join(email_body).replace('= ', '').replace('=20', '')
                        if '@' in email_body:
                            extract_email_pattern = r"\S+@\S+\.\S+"
                            listed_emails = re.findall(extract_email_pattern, email_body) # returns ['hello@uibakery.io', 'contact@uibakery.io']
                            for email in listed_emails:
                                email_body = email_body.replace(email, '')
                        if '---' in email_body:
                            email_body = email_body.split('---')[0]
                        if '***' in email_body:
                            email_body = email_body.split('***')[0]
                        if '___' in email_body:
                            email_body = email_body.split('___')[0]
                        if '===' in email_body:
                            email_body = email_body.split('===')[0]
                        data['Body'] = email_body.replace('[IMAGE]','').replace('\t','')
                        inbox_email_data.append(data)
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    print(e)
                    print(f'{field_name}: {field_value}')

            inbox_df = pd.DataFrame(inbox_email_data)
        if 'sent' in folders:
            sent_path = employee_path+'sent/'
            sent_emails = next(os.walk(sent_path))[2]
            sent_email_data = []
            for file in sent_emails:
                data = {'Employee': employee, 'Type':'sent'}
                try:
                    with open(f'{sent_path}{file}', 'r') as file:
                        raw_email = file.read()
                        sections = raw_email.split('\n')
                        email_body = []
                        for section in sections:
                            if ': ' in section:
                                field_name = section.split(': ')[0]
                                field_value = section.split(': ')[1]
                                if field_name == 'Date':
                                    data[field_name] = field_value
                                elif field_name == 'X-To':
                                    if '<' in field_value:
                                        sender_name_backwards = field_value.split('<')[0]
                                        if len(sender_name_backwards) == 0:
                                            sender_name = field_value.split('<')[1].split('>')[0]
                                        elif ', ' in sender_name_backwards:
                                            sender_name_list = sender_name_backwards.split(', ')
                                            sender_name = sender_name_list[1] + sender_name_list[0]
                                        elif ',' in sender_name_backwards:
                                            sender_name_list = sender_name_backwards.split(',')
                                            sender_name = sender_name_list[1] + sender_name_list[0]
                                        else:
                                            sender_name = sender_name_backwards
                                        data['To'] = sender_name.replace('"','')
                                    else:
                                        data['To'] = field_value
                                elif field_name == 'To':
                                    if 'To' in data.keys():
                                        pass
                                    else:
                                        sender = field_value.split('@')[0]
                                        if len(sender.split('.')) > 1:
                                            sender_name = sender.split('.')[0].capitalize() + sender.split('.')[1].capitalize()
                                        else:
                                            first_initial = sender[0].upper()
                                            sender_name = first_initial + sender[1:].capitalize()
                                        data['To'] = sender_name
                                elif field_name == 'Subject':
                                    data['Subject'] = field_value
                                else:
                                    continue
                            else:
                                email_body.append(section)
                        email_body = (' ').join(email_body).replace('= ', '').replace('=20', '')
                        if '@' in email_body:
                            extract_email_pattern = r"\S+@\S+\.\S+"
                            listed_emails = re.findall(extract_email_pattern, email_body) # returns ['hello@uibakery.io', 'contact@uibakery.io']
                            for email in listed_emails:
                                email_body = email_body.replace(email, '')
                        if '---' in email_body:
                            email_body = email_body.split('---')[0]
                        if '***' in email_body:
                            email_body = email_body.split('***')[0]
                        if '___' in email_body:
                            email_body = email_body.split('___')[0]
                        if '===' in email_body:
                            email_body = email_body.split('===')[0]
                        data['Body'] = email_body.replace('[IMAGE]','').replace('\t','')
                        sent_email_data.append(data)
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    print(e)
            sent_df = pd.DataFrame(sent_email_data)
        employee_dict = {'Employee':employee, 'Inbox':inbox_df,'Sent':sent_df}
        emails.append(employee_dict)
    return emails

emails = process_emails(data_path)

In [3]:
display(emails[0]['Inbox'].head())

,Employee,Type,Date,From,Subject,Body
0,arnold-j,received,"Thu, 25 Oct 2001 09:18:50 -0700 (PDT)",Kimberly Banner,hi,Hey there. We are going to be at Front Porch...
1,arnold-j,received,"Tue, 27 Nov 2001 10:25:00 -0800 (PST)",Kelly Busch,keeping the lights on,John- Was lovely meeting you this weekend. ...
2,arnold-j,received,"Mon, 31 Dec 2001 13:20:07 -0800 (PST)",customerservice@qwikfliks.com,L.A. Confidential has been received.,Qwiklist(9) MyAccount Rental History ...
3,arnold-j,received,"Tue, 20 Nov 2001 06:57:42 -0800 (PST)",Tanya Rohauer,AIG positions,"John, As discussed, the AIG exposure is $57M..."
4,arnold-j,received,"Tue, 1 Jan 2002 11:04:18 -0800 (PST)",Mark Sagel,Market update,Happy New Year and best wishes. - Energy010...
